In [94]:
# Modificar
principal = 'C2 CENTRO'						# C2 a analizar
archivo = 'Folios_boton_promad_2023-03-26'	# No incluir la extensión
DATA_PATH = '../data/'							# Carpeta donde se encuentran los archivos

# No modificar
	# C2
C2 = ['C2 CENTRO', 'C2 NORTE', 'C2 ORIENTE', 'C2 PONIENTE', 'C2 SUR']
	# Directorios
RESOURCES_PATH = DATA_PATH+archivo+'.csv'
GRAPHICS_PATH = DATA_PATH+'Graficos_'+"_".join(archivo.split('_')[1:])+'.xlsx'

### Configuracion Inicial

In [95]:
# Install xlsxwriter & openpyxl if you don't have it
# !pip install xlsxwriter openpyxl

import pandas as pd
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl import Workbook
from openpyxl.chart import BarChart, Series, Reference
from openpyxl.chart.marker import DataPoint
from openpyxl.chart.label import DataLabelList
from openpyxl.styles import PatternFill, Border, Side, Font, Alignment, Color
from openpyxl.formatting.rule import ColorScale, FormatObject, ColorScaleRule
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.utils.dataframe import dataframe_to_rows


In [96]:
def apply_to_range(ws:Worksheet, cells_range:str, style:dict, border_cell:bool=False):
	"""
	Apply styles to a range of cells in a worksheet
	:param ws: Worksheet
	:param range: Range of cells to apply the styles
	:param style: Dictionary with the styles to apply
			style sample: {'font': Font(name='Calibri', size=11, bold=True, color='FF000000'),
	"""
	# Get the first and last cell
	first_cell, last_cell = cells_range.split(':')
	# Get the first and last column
	first_col, first_row = first_cell[0], int(first_cell[1:])
	last_col, last_row = last_cell[0], int(last_cell[1:])

	# Iterate over the range
	for row in range(first_row, last_row+1):
		for col in range(ord(first_col), ord(last_col)+1):
			# Get the cell
			cell = ws[chr(col)+str(row)]
			# Apply the styles
			for key, value in style.items():
				if key == 'border' and not border_cell:
					bleft = None; bright = None; btop = None; bbottom = None
					if value.left	and cell.column_letter==first_col:
						bleft = value.left
						# cell.border.left	= value.left
					if value.right	and cell.column_letter==last_col:
						bright = value.right
						# cell.border.right	= value.right
					if value.top	and cell.row==first_row:
						btop = value.top
						# cell.border.top		= value.top
					if value.bottom	and cell.row==last_row:
						bbottom = value.bottom
						# cell.border.bottom	= value.bottom
					cell.border = Border(left=bleft, right=bright, top=btop, bottom=bbottom)
				else:
					setattr(cell, key, value)

# Function save a pandas dataframe to an excel file using workbook and worksheet without ExcelWriter
# Dont use pd.ExcelWriter only use openpyxl
def save_df_to_excel(df:pd.DataFrame, ws:Worksheet, startrow=None, startcol=None):
	"""
	Save a pandas dataframe to an excel file using workbook and worksheet without ExcelWriter
	:param df: Pandas dataframe to save
	:param ws: Worksheet
	:param startrow: Start row to save the dataframe
	:param startcol: Start column to save the dataframe
	"""

	# Write the columns
	if startrow is None:
		startrow = 1
	if startcol is None:
		startcol = 1

	rows = dataframe_to_rows(df, index=True, header=True)
	for r_idx, row in enumerate(rows, start=startrow):
		for c_idx, value in enumerate(row, start=startcol):
			ws.cell(row=r_idx, column=c_idx, value=value)

In [97]:
# Get the workbook
wb = Workbook()
wb.remove(wb.active)

# Create a sample dataframe
data = pd.read_csv(RESOURCES_PATH, encoding='latin-1')
data.head()

,FOLIO,FECHA_INICIO,HORA_INICIO,FECHA_CIERRE,HORA_CIERRE,TIPO_INCIDENTE_INICIO,CLASIFICACION_INCIDENTE_CIERRE,TIPO_INCIDENTE_CIERRE,ORIGEN,C2_INICIO,...,CODIGO_CIERRE,dia_cierre,semana_cierre,mes_cierre,USUARIO_CREA_INCIDENTE,USUARIO_INTERVENCION,USUARIO_MOVILIZA,USUARIO_CIERRE_INCIDENTE,sin_corregir,reclasificados
0,C2P/20230101/00154,01/01/2023,19:23:31,2023-01-01,22:33:21,BOTON DE AUXILIO,SERVICIO,FALSA ALARMA-BOTÓN DE AUXILIO,BOTON DE AUXILIO,C2 PONIENTE,...,CM,domingo,52,Enero,EXTERNO,CP902225,-,JOBSISTEMA,Corregido,Reclasificado-otro
1,C2P/20230101/00149,01/01/2023,19:06:20,2023-01-01,20:18:42,BOTON DE AUXILIO,URGENCIAS MEDICAS,LESIONADO-POR GOLPES-AGRESION,BOTON DE AUXILIO,C2 PONIENTE,...,ASI,domingo,52,Enero,EXTERNO,C5822259,CP891581,C5822259,Corregido,Reclasificado-otro
2,C2P/20230101/00150,01/01/2023,19:07:22,2023-01-01,22:13:30,BOTON DE AUXILIO,SERVICIO,SERVICIOS-ENTREVISTA CIUDADANA,BOTON DE AUXILIO,C2 PONIENTE,...,ASI,domingo,52,Enero,EXTERNO,CP764023,CP764023,JOBSISTEMA,Corregido,Reclasificado-otro
3,C2P/20230101/00141,01/01/2023,18:17:13,2023-01-01,19:05:11,BOTON DE AUXILIO,DELITO,DISTURBIO-RIÑA,BOTON DE AUXILIO,C2 PONIENTE,...,D,domingo,52,Enero,EXTERNO,CP764023,CP764023,CP764023,Corregido,Reclasificado-otro
4,C2C/20230101/00056,01/01/2023,04:44:19,2023-01-01,07:53:20,BOTON DE AUXILIO,SERVICIO,FALSA ALARMA-BOTÓN DE AUXILIO,BOTON DE AUXILIO,C2 CENTRO,...,CM,domingo,52,Enero,EXTERNO,CC1112453,-,JOBSISTEMA,Corregido,Reclasificado-otro


### Tabla - No Reclasificados

In [98]:
""" 
1. Generate a Pivot Table with the following structure:
MultiIndex([(    'Diciembre',    'Corregido'),
            (    'Diciembre', 'Sin corregir'),
            (    'Diciembre',        'Total'),
            (        'Enero',    'Corregido'),
            (        'Enero', 'Sin corregir'),
            (        'Enero',        'Total'),
            (      'Febrero',    'Corregido'),
            (      'Febrero', 'Sin corregir'),
            (      'Febrero',        'Total'),
            ('Total General',             '')],
           names=['mes_cierre', 'sin_corregir'])
	   
2. Then, Generate a pivot table with the following structure:
+---------------+------------+------------+-------------+--------------+------+------+
| TOTAL                                                 | SIN RECLASIFICAR           |
+---------------+------------+------------+-------------+--------------+------+------+
| C2              ENE          FEB          MAR           ENE            FEB    MAR  |
+---------------+------------+------------+-------------+--------------+------+------+
| C2 CENTRO       5,429        3,828        2,883         1.8%           2.7%   1.4% |
| C2 NORTE        4,497        3,824        3,291         2.5%           3.3%   2.1% |
+===============+============+============+=============+==============+======+======+
|| C2 ORIENTE     4,977        4,341        3,362         3.3%           6.4%   3.3% ||		'Cuz principal=='C2 ORIENTE'
+===============+============+============+=============+==============+======+======+
| C2 PONIENTE     3,962        3,763        3,448         1.5%           1.6%   1.3% |
| C2 SUR          3,207        2,923        2,464         2.7%           2.9%   1.3% |
| Total General   22,072       18,679       15,448        2.3%           2.8%   1.9% |
+---------------+------------+------------+--------------+--------------+------+------+

"""
pivot_table = pd.pivot_table(data[data['CODIGO_MOTIVO_CIERRE_INSTITUCION'] != 'SYS'], 
                             values='FOLIO', aggfunc='count',
                             index='C2_INICIO', 
                             columns=['mes_cierre', 'sin_corregir'], 
                             margins=True, margins_name='Total General')


# Rename Sin corregir to SIN CLASIFICAR and Corregido to CLASIFICADO
pivot_table.rename(columns={'Sin corregir': 'SIN RECLASIFICAR', 'Corregido':'CLASIFICADO'}, level=1, inplace=True)
# Rename C2_INICIO to C2
pivot_table.rename_axis('C2', axis=0, inplace=True)
# UPPER CASE and 3 CHARACTERS to all Columns 1lvl, except Total General
pivot_table.columns = pd.MultiIndex.from_tuples([(x[0].upper()[:3], x[1]) if x[0] != 'Total General' else (x[0], x[1]) for x in pivot_table.columns])

# Sum all columns except Total General
for col in pivot_table.columns.get_level_values(0).unique()[:-1]:	
	if col != 'Total General':	# Except Total General
		pivot_table[col, 'TOTAL'] = pivot_table[col].sum(axis=1)
		# Convert to percentage all second level columns except TOTAL
		for col2 in pivot_table[col].columns:
			if col2 != 'TOTAL':
				pivot_table[col, col2] = pivot_table[col, col2]/pivot_table[col, 'TOTAL']
				# pivot_table[col, col2] = pivot_table[col, col2].apply(lambda x: f'{x:.1%}')


# Invert order levels
pivot_table = pivot_table.swaplevel(0, 1, axis=1)
# First TOTAL, then CLASIFICADO, then SIN CLASIFICAR
pivot_table = pivot_table.reindex(columns=['TOTAL', 'CLASIFICADO', 'SIN RECLASIFICAR'], level=0)

print(pivot_table)

# Order columns
pivot_table = pivot_table[pivot_table.columns.get_level_values(0).unique()]

# Drop CLASIFICADO column
pivot_table = pivot_table.drop(columns='CLASIFICADO', level=0)

""" 
	Save Table
"""
ws = wb.create_sheet('Tabla - No Clasificados')
save_df_to_excel(pivot_table, ws)

"""
	Formatting
"""

ws['A1'] = 'TOTAL'
# Merge the cells
ws.merge_cells('A1:D1')
ws.merge_cells('E1:G1')
# Move A3 to A2 
ws.move_range('A3', rows=-1, cols=0)
# Move A4:G9 to A3:G8
ws.move_range('A4:G9', rows=-1, cols=0)


total_fill 			= PatternFill(start_color="215967", end_color="215967", fill_type="solid")
sinreclasificar_fill= PatternFill(start_color="963634", end_color="963634", fill_type="solid")
c2_fill 			= PatternFill(start_color="d9d9d9", end_color="d9d9d9", fill_type="solid")
c2resaltado_fill	= PatternFill(start_color="ffffff", end_color="ffffff", fill_type="solid")
white_font			= Font(color="ffffff", bold=True, size=11, condense=False)
c2_font				= Font(color="595959", bold=False, size=11, condense=False)
c2resaltado_font	= Font(color="000000", bold=False, size=14, condense=False)
black_border		= Border(left=Side(style='medium', color='000000'),
                    		 right=Side(style='medium', color='000000'),
                    		 top=Side(style='medium', color='000000'),
                    		 bottom=Side(style='medium', color='000000'))

#ws['A1:D2'].fill = total_fill
apply_to_range(ws, 'A1:G8', {
	'fill': total_fill,
	'font': white_font,
	'border': Border(left=None, right=None, top=None, bottom=None),
	'alignment': Alignment(horizontal='center', vertical='center'),
})
# ws['E1:G2'].fill = sinreclasificar_fill
apply_to_range(ws, 'E1:G2', {
	'fill': sinreclasificar_fill,
})
# ws['A3:G7'].fill = c2_fill
apply_to_range(ws, 'A3:G7', {
	'fill': c2_fill,
	'font': c2_font,
	'alignment': Alignment(horizontal='left', vertical='center'),
})
# ws['B3:G7'].number_format = '#,###'
apply_to_range(ws, 'B3:G8', {
	'number_format': '#,###',
	'alignment': Alignment(horizontal='center', vertical='center'),
})
# ws['E3:G7'].number_format = '0.0%'
apply_to_range(ws, 'E3:G8', {
	'number_format': '0.0%',
	'alignment': Alignment(horizontal='center', vertical='center'),
})

i = 3+C2.index(principal)
apply_to_range(ws, f'A{i}:G{i}', {
	'fill': c2resaltado_fill,
	'font': c2resaltado_font,
	'border': black_border,
})

ws.column_dimensions['A'].width = 36
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 20
ws.column_dimensions['D'].width = 20
ws.column_dimensions['E'].width = 16
ws.column_dimensions['F'].width = 16
ws.column_dimensions['G'].width = 16

               TOTAL               CLASIFICADO                      \
                 ENE    FEB    MAR         ENE       FEB       MAR   
C2                                                                   
C2 CENTRO       5429   3828   3621    0.982133  0.973354  0.981773   
C2 NORTE        4497   3824   4042    0.975095  0.967050  0.980208   
C2 ORIENTE      4977   4341   4566    0.967450  0.936420  0.967148   
C2 PONIENTE     3963   3763   4443    0.985112  0.984055  0.988521   
C2 SUR          3207   2923   3323    0.972872  0.970578  0.987361   
Total General  22073  18679  19995    0.976578  0.965202  0.980545   

              SIN RECLASIFICAR                      
                           ENE       FEB       MAR  
C2                                                  
C2 CENTRO             0.017867  0.026646  0.018227  
C2 NORTE              0.024905  0.032950  0.019792  
C2 ORIENTE            0.032550  0.063580  0.032852  
C2 PONIENTE           0.014888  0.015945  0.011479 

### Tabla - Códigos de Cierre

In [99]:
""" 
1. Generate a Pivot Table
2. Then, Generate a pivot table with the following structure:
+------------------+----------+---------+---------+-------+
|                  | Sin reclasificar             |       |
+ Código de cierre +----------+---------+---------+ TOTAL +
|                  | ENE      | FEB     | MAR     |       |
+------------------+----------+---------+---------+-------+
| ACI              | 13       | 16      | 7       | 36    |
| ASI              | 21       | 16      | 14      | 51    |
| D                | 30       | 42      | 11      | 83    |
| F                | 13       | 9       | 2       | 24    |
| I                | 10       | 3       | 8       | 21    |
+------------------+----------+---------+---------+-------+
| SUB TOTAL        | 87       | 86      | 42      | 215   |	
+------------------+----------+---------+---------+-------+
| SYS              | 131      | 131     | 86      | 325   |
+------------------+----------+---------+---------+-------+
| TOTAL            | 218      | 218     | 128     | 540   |
+------------------+----------+---------+---------+-------+
"""

pivot_table = pd.pivot_table(data[data['C2_INICIO'] == principal],
                             values=['FOLIO'], aggfunc='count', 
                             index=['CODIGO_MOTIVO_CIERRE_INSTITUCION'], 
                             columns=['mes_cierre', 'sin_corregir'], 
                             fill_value=0,
                             margins=True, margins_name='TOTAL')


print(pivot_table.columns)
# Delete first level of columns


# Rename C2_INICIO to C2
pivot_table.rename_axis('Código de Cierre', axis=0, inplace=True)
# UPPER CASE and 3 CHARACTERS to all Columns 1lvl, except Total General
pivot_table.columns = pd.MultiIndex.from_tuples([(x[1].upper()[:3], x[2]) if x[1] != 'TOTAL' else (x[2], x[1]) for x in pivot_table.columns])
# Rename Sin corregir to SIN CLASIFICAR and Corregido to CLASIFICADO
pivot_table.rename(columns={'Sin corregir': 'SIN CLASIFICAR', 'Corregido':'CLASIFICADO'}, level=1, inplace=True)


# Invert order levels
pivot_table = pivot_table.swaplevel(0, 1, axis=1)
# First TOTAL, then CLASIFICADO, then SIN CLASIFICAR
pivot_table = pivot_table.reindex(columns=['CLASIFICADO', 'SIN CLASIFICAR', 'TOTAL'], level=0)
# Drop CLASIFICADO column
pivot_table = pivot_table.drop(columns='CLASIFICADO', level=0)

# Sum all rows except SYS and TOTAL and save the sum in 'SUB TOTAL' (before SYS)
pivot_table.loc['SUB TOTAL'] = pivot_table.loc[~pivot_table.index.isin(['SYS', 'TOTAL'])].sum()

# Sum all columns except TOTAL and update column TOTAL
for col in pivot_table.columns.get_level_values(0).unique()[:-1]:
	if col != 'TOTAL':	# Except Total General
		pivot_table['TOTAL'] = pivot_table[col].sum(axis=1)

# Place SUB TOTAL before SYS and after the rest
ordered_index = list(pivot_table.index[:-3]) + ['SUB TOTAL', 'SYS', 'TOTAL']
pivot_table = pivot_table.reindex(ordered_index)

pivot_table

""" 
	Save Table without format
"""
# Write the pivot table to the new excel file
# pivot_table.to_excel(writer, sheet_name='Tabla - Código de Cierre', startrow=0, startcol=0)

"""
	Formatting
"""
# Get the worksheet
# ws = writer.sheets['Tabla - Código de Cierre']
ws = wb.create_sheet('Tabla - Código de Cierre')
save_df_to_excel(pivot_table, ws)

j = len(pivot_table)+2

ws.move_range('A3', rows=-2, cols=0)
ws.move_range(f'A4:E{j+1}', rows=-1, cols=0)
ws.merge_cells('A1:A2')
ws.merge_cells('B1:D1')
ws.merge_cells('E1:E2')

base_fill			= PatternFill(fill_type="solid", start_color="215968", end_color="215968")
content_fill		= PatternFill(fill_type="solid", start_color="b7dee8", end_color="b7dee8")
sys_fill			= PatternFill(fill_type="solid", start_color="808080", end_color="808080")
total_fill			= PatternFill(fill_type="solid", start_color="31869b", end_color="31869b")
subtotal_total_fill	= PatternFill(fill_type="solid", start_color="da9694", end_color="da9694")
total_total_fill	= PatternFill(fill_type="solid", start_color="963634", end_color="963634")


white_font			= Font(color="ffffff", bold=True, size=11, condense=False)
default_font		= Font(color="000000", bold=False, size=11, condense=False)

black_border		= Border(left	= Side(style='thin', color='000000'),
                    		 right	= Side(style='thin', color='000000'),
                    		 top	= Side(style='thin', color='000000'),
                    		 bottom	= Side(style='thin', color='000000'))

# color_scale = ColorScale(cfvo=[FormatObject(type='min'), FormatObject(type='max')], 
# 			 			 color=[Color('daeef3'), Color('31869b')])

color_rule = ColorScaleRule(start_type='min', start_color='daeef3',
			    			end_type='max', end_color='31869b')

apply_to_range(ws, f'A1:E{j}', {
	'fill': base_fill,
	'font': white_font,
	'border': black_border,
	'alignment': Alignment(horizontal='center', vertical='center', wrapText=True),
}, True)
apply_to_range(ws, 'B2:D2', {
	'border': black_border
})
apply_to_range(ws, f'B3:E{j}', {
	'number_format': '#,##0',
})
apply_to_range(ws, f'B3:D{j-3}', {
	'fill': content_fill,
	'font': default_font,
	'border': black_border
})
apply_to_range(ws, f'E3:E{j-3}', {
	'fill': content_fill,
	'font': default_font,
	'border': black_border
}, True)
apply_to_range(ws, f'A{j-1}:E{j-1}', {
	'fill': sys_fill,
})
apply_to_range(ws, f'A{j}:D{j}', {
	'fill': total_fill,
})
apply_to_range(ws, f'E{j-2}:E{j-2}', {
	'fill': subtotal_total_fill,
})
apply_to_range(ws, f'E{j}:E{j}', {
	'fill': total_total_fill,
})

ws.conditional_formatting.add(f'E3:E{j-3}', color_rule)

ws.column_dimensions['A'].width = 13
ws.column_dimensions['B'].width = 12
ws.column_dimensions['C'].width = 12
ws.column_dimensions['D'].width = 12
ws.column_dimensions['E'].width = 12
ws.row_dimensions[j-2].height = 30
ws.row_dimensions[j].height = 30

MultiIndex([('FOLIO',   'Enero',    'Corregido'),
            ('FOLIO',   'Enero', 'Sin corregir'),
            ('FOLIO', 'Febrero',    'Corregido'),
            ('FOLIO', 'Febrero', 'Sin corregir'),
            ('FOLIO',   'Marzo',    'Corregido'),
            ('FOLIO',   'Marzo', 'Sin corregir'),
            ('FOLIO',   'TOTAL',             '')],
           names=[None, 'mes_cierre', 'sin_corregir'])


### Grafica - Códigos de Cierre

In [100]:
""" 
1. Generate a Pivot Table with the following structure:
MultiIndex([(   'Corregido'),
            ('Sin corregir'),
            (      'Total ')],
           names=['c2_incio', 'sin_corregir'])
	   
2. Then, Generate a pivot table with the following structure:

+---------------+--------------+--------------+---------------+
| C2              Corregido      Sin Corregir   Total         |
+---------------+--------------+--------------+---------------+
| C2 CENTRO       5,429          3,828          2,883         |
| C2 NORTE        4,497          3,824          3,291         |
| C2 PONIENTE     3,962          3,763          3,448         |
| C2 SUR          3,207          2,923          2,464         |
| Total           22,072         18,679         15,448        |
+---------------+--------------+--------------+---------------+

"""
pivot_table = pd.pivot_table(data[data['CODIGO_MOTIVO_CIERRE_INSTITUCION'] != 'SYS'], 
                             values='FOLIO', aggfunc='count',
                             index='C2_INICIO', 
                             columns=['sin_corregir'], 
                             margins=True, margins_name='Total General')


# Rename Sin corregir to SIN CLASIFICAR and Corregido to CLASIFICADO
pivot_table.rename(columns={'Sin corregir': 'SIN RECLASIFICAR', 'Corregido':'CLASIFICADO'}, inplace=True)
# Rename Columns name to C2
pivot_table.columns.names = ['C2']

# pivot_table.to_excel(writer, sheet_name='Gráfica - No Clasificados', startrow=1, startcol=1)
# ws = writer.sheets['Gráfica - No Clasificados']
ws = wb.create_sheet('Gráfica - No Clasificados')
save_df_to_excel(pivot_table, ws)

"""
	Formatting
"""
j = len(pivot_table)+1

ws.move_range('A2', rows=-1, cols=0)
ws.move_range(f'A3:D{j+1}', rows=-1, cols=0)

""" 
	Generate clustered column chart with the pivot table
"""
data = Reference(ws, min_col=2, max_col=3, min_row=1, max_row=j-1)
cats = Reference(ws, min_col=1, max_col=1, min_row=2, max_row=j-1)

chart = BarChart()
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)

chart.type = "col"
chart.grouping = "stacked"
chart.overlap = 100

series = chart.series

series[0].graphicalProperties.line.noFill = True
series[0].graphicalProperties.solidFill = "c3d69b"

dp = DataPoint(idx=3)
dp.graphicalProperties.line.noFill = True
dp.graphicalProperties.solidFill = "77933c"
series[0].dPt.append(dp)

datalabels = DataLabelList()
datalabels.showVal = True
datalabels.position = "ctr"
datalabels.numFmt = "#,##0"
# # TODO https://openpyxl.readthedocs.io/en/latest/charts/graphical.html#reusing-xml
# datalabels.txPr = RichText(
#     properties=TextProperties(bold=True, sz=1000, baseline=0, kern=1200, spc=0, normalizeH=1, noProof=1, latin=Latin(font="Calibri", typeface="+mn-lt"), ea=EastAsian(font="Calibri", typeface="+mn-ea"), cs=ComplexScript(font="Calibri", typeface="+mn-cs")),	
# )
# chart.dataLabels = datalabels





series[1].graphicalProperties.line.noFill = True
series[1].graphicalProperties.solidFill = "953735"

dp = DataPoint(idx=3)
dp.graphicalProperties.line.noFill = True
dp.graphicalProperties.solidFill = "d99694"
series[1].dPt.append(dp)



ws.add_chart(chart, "G2")

# # UPPER CASE and 3 CHARACTERS to all Columns 1lvl, except Total General
# pivot_table.columns = pd.MultiIndex.from_tuples([(x[0].upper()[:3], x[1]) if x[0] != 'Total General' else (x[0], x[1]) for x in pivot_table.columns])

# # Sum all columns except Total General
# for col in pivot_table.columns.get_level_values(0).unique()[:-1]:	
# 	if col != 'Total General':	# Except Total General
# 		pivot_table[col, 'TOTAL'] = pivot_table[col].sum(axis=1)
# 		# Convert to percentage all second level columns except TOTAL
# 		for col2 in pivot_table[col].columns:
# 			if col2 != 'TOTAL':
# 				pivot_table[col, col2] = pivot_table[col, col2]/pivot_table[col, 'TOTAL']
# 				# pivot_table[col, col2] = pivot_table[col, col2].apply(lambda x: f'{x:.1%}')


# # Invert order levels
# pivot_table = pivot_table.swaplevel(0, 1, axis=1)
# # First TOTAL, then CLASIFICADO, then SIN CLASIFICAR
# pivot_table = pivot_table.reindex(columns=['TOTAL', 'CLASIFICADO', 'SIN RECLASIFICAR'], level=0)


# # Order columns
# pivot_table = pivot_table[pivot_table.columns.get_level_values(0).unique()]

# # Drop CLASIFICADO column
# pivot_table = pivot_table.drop(columns='CLASIFICADO', level=0)

print(pivot_table)

C2             CLASIFICADO  SIN RECLASIFICAR  Total General
C2_INICIO                                                  
C2 CENTRO            12613               265          12878
C2 NORTE             12045               318          12363
C2 ORIENTE           13296               588          13884
C2 PONIENTE          11999               170          12169
C2 SUR                9238               215           9453
Total General        59191              1556          60747


In [101]:
# Save the excel file
wb.save(GRAPHICS_PATH)
# wb.close()
# writer.close()